In [ ]:
#default_exp logger

In [ ]:
#export
import json
import os
from datetime import datetime as dt
import boto3
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute

In [ ]:
#export
class PynamoLogger(Model):
  ''' log changes to pynamo member '''
  class Meta:
    table_name = None
    region = 'ap-southeast-1'
  appName = UnicodeAttribute(hash_key = True)
  timestamp = NumberAttribute(range_key=True)
  logMessage = UnicodeAttribute()
  requestObject = JSONAttribute( default = {'noValue': 'noValue'})
  responseObject = JSONAttribute(default = {'noValue': 'noValue'})
  def to_dict(self):
    res = {}
    for k in ['appName', 'timestamp', 'logMessage', 'requestObject', 'responseObject']:
      res[k] = getattr(self, k)
    return res  
  @classmethod
  def log(cls, message:str, appName = 'mockapp', 
          requestObject = {'request': "none"}, responseObject = {'message': 'not specified'}):
    dynamoLogger = cls(
        appName = appName,
        timestamp = dt.now().timestamp(),
        logMessage = message,
        responseObject = responseObject
    )
    saveResult =  dynamoLogger.save()
    return {'logId': dynamoLogger.timestamp, 'saveResult': saveResult}

  @classmethod
  def checkLog(cls, appName = 'mockapp', logId = '', logType = 'write', limit = 10):
    return [log.to_dict() for log in cls.query(
      hash_key = appName,
      range_key_condition = cls.timestamp == float(logId) if logId else None,
      limit = limit
    )]
    
      


# Usage

In [ ]:
class Logger(PynamoLogger):
  class Meta:
    table_name = 'member-database-log-dev-manual'
    region = 'ap-southeast-1'
    billing_mode = 'PAY_PER_REQUEST'
    

In [ ]:
# Logger.create_table()

In [ ]:
Logger.log(appName = 'test', message = "this is crazy", responseObject = {'response': 'null response'})

{'logId': 1603075540.846738,
 'saveResult': {'ConsumedCapacity': {'CapacityUnits': 1.0,
   'TableName': 'member-database-log-dev-manual'}}}

In [ ]:
Logger.checkLog(appName = 'test', logId = '1603074235.103042')

[{'appName': None,
  'timestamp': 1603074235.103042,
  'logMessage': 'this is crazy',
  'requestObject': {'noValue': 'noValue'},
  'responseObject': {'response': 'null response'}}]